In [34]:
import numpy as np
import scipy.sparse as sp
import time

import igl
import meshplot as mp

In [35]:
def benchmark(f, repeat=10, *args, **kwargs):
    times = []

    for _ in range(repeat):
        tic = time.time()
        f(*args, **kwargs)
        toc = time.time()
        times.append(toc - tic)

    times = np.array(times)

    print(f"repeat {repeat}")
    # for t in times:
    #     print(f"time {t:.4f}s")

    print()
    print(f"average {np.mean(times):.8f}s")
    print(f"minimum {np.min(times):.8f}s")
    print(f"maximum {np.max(times):.8f}s")
    print(f"std dev {np.std(times):.8f}s")

# Part1. Read DATA (obj, off, ...)

In [36]:
# V, F = igl.read_triangle_mesh("data/sphere.obj")
# V, F = igl.read_triangle_mesh("data/truck.obj")
V, F = igl.read_triangle_mesh("data/human.obj")

In [37]:
V.shape[0]

24461

In [38]:
F.shape[0]

48918

# Part2. Compute normals for vertices

In [39]:
nor1_vec = np.array([0.57735027, 0.57735027, 0.57735027])
ni_face = igl.per_face_normals(V, F, nor1_vec)

In [40]:
ni_vertex = igl.per_vertex_normals(V, F)

In [41]:
# ni_face[:2, :]

In [43]:
ni_vertex[77:82, :]

array([[-0.70531118, -0.34799726, -0.61760347],
       [-0.50873144, -0.34111434, -0.790464  ],
       [-0.55473048, -0.30656063, -0.77349511],
       [-0.43593887, -0.31290787, -0.84382816],
       [-0.44070444, -0.2582792 , -0.85969265]])

# Part3. Compute adjacency triangles for each face

In [30]:
# -1 means no adj triangle
tt, _ = igl.triangle_triangle_adjacency(F)

In [31]:
# tt

# Part4. Compute mass matrix for each vi

In [32]:
mass = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# mass = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)

In [33]:
mass

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 382 stored elements and shape (382, 382)>

In [34]:
mass.data[:5]

array([0.00376191, 0.00376197, 0.00376196, 0.00376192, 0.00376203])

# Part5. Compute cotangent matrix for vi - vi

In [161]:
ce = igl.cotmatrix_entries(V, F)

In [162]:
ce.shape

(48918, 3)

In [163]:
cot = igl.cotmatrix(V, F)

In [164]:
cot

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 171215 stored elements and shape (24461, 24461)>

In [165]:
coo = cot.tocoo()

In [166]:
coo.row

array([    0,     1,     2, ..., 21867, 21870, 24460], dtype=int32)

In [167]:
coo.col

array([    0,     0,     0, ..., 24460, 24460, 24460], dtype=int32)

In [168]:
coo.data[:6]

array([-3.73435332,  0.72395803,  0.33445503,  0.7887416 ,  0.77900912,
        0.11443824])

# BenchMark

In [25]:
def bench_perfacenormal():
    nor1_vec = np.array([0.57735027, 0.57735027, 0.57735027])
    ni_face = igl.per_face_normals(V, F, nor1_vec)

def bench_pervertexnormal():
    ni_vertex = igl.per_vertex_normals(V, F)

def bench_mass():
    mass = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)

def compute_cot():
    igl.cotmatrix(V, F)

def compute_cot_entries():
    igl.cotmatrix_entries(V, F)

In [26]:
benchmark(bench_pervertexnormal, repeat=100)

repeat 100

average 0.00089823s
minimum 0.00048208s
maximum 0.00336194s
std dev 0.00049036s
